### Import libraries





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pip install category_encoders

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from pandas_profiling import ProfileReport
%matplotlib inline



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Import dataset 

In [ ]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

In [ ]:
X_train_transformed = df_train.drop(['RainTomorrow'], axis=1)
y_train = df_train['RainTomorrow']
X_test_transformed = df_test.drop(['RainTomorrow'], axis=1)
y_test = df_test['RainTomorrow']

### Declare feature vector and target variable <a class="anchor" id="8"></a>

In [ ]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']
X_drop_miss = df_drop_miss.drop(['RainTomorrow'], axis=1)
y_drop_miss = df_drop_miss['RainTomorrow']
X_drop_relate = df_drop_relate.drop(['RainTomorrow'], axis=1)
y_drop_relate = df_drop_relate['RainTomorrow']

### Split data into separate training and test set <a class="anchor" id="9"></a>

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train_drop_miss, X_test_drop_miss, y_train_drop_miss, y_test_drop_miss = train_test_split(X_drop_miss, y_drop_miss, test_size = 0.2, random_state = 0, stratify = y_drop_miss)
X_train_drop_relate, X_test_drop_relate, y_train_drop_relate, y_test_drop_relate = train_test_split(X_drop_relate, y_drop_relate, test_size = 0.2, random_state = 0, stratify = y_drop_relate)
X_train_big, X_test_big, y_train_big, y_test_big = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(X_train_big, y_train_big, test_size=0.3, random_state =0,stratify=y_train_big)
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(X_test_big, y_test_big, test_size=0.2, random_state =0,stratify=y_test_big)

In [ ]:
# check the shape of X_train and X_test
print(X_train_big.shape, X_test_big.shape)
print(X_train_medium.shape, X_test_medium.shape)
print(X_train_small.shape, X_test_small.shape)

(113754, 24) (28439, 24)
(79627, 24) (34127, 24)
(22751, 24) (5688, 24)


In [ ]:
def FE(X_train,X_test, y_train, y_test):
  categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']
  numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']
  for df1 in [X_train, X_test]:
    for col in numerical:
        col_median=X_train[col].median()
        df1[col].fillna(col_median, inplace=True)  
  for df2 in [X_train, X_test]:
    df2['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    df2['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    df2['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    df2['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)
  encoder = ce.BinaryEncoder(cols=['RainToday'])
  X_train = encoder.fit_transform(X_train)
  X_test = encoder.transform(X_test)
  X_train = pd.concat([X_train[numerical], X_train[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_train.Location), 
                     pd.get_dummies(X_train.WindGustDir),
                     pd.get_dummies(X_train.WindDir9am),
                     pd.get_dummies(X_train.WindDir3pm)], axis=1)
  X_test = pd.concat([X_test[numerical], X_test[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_test.Location), 
                     pd.get_dummies(X_test.WindGustDir),
                     pd.get_dummies(X_test.WindDir9am),
                     pd.get_dummies(X_test.WindDir3pm)], axis=1)
  cols = X_train.columns
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  X_train = pd.DataFrame(X_train, columns=[cols])
  X_test = pd.DataFrame(X_test, columns=[cols])
  return X_train, y_train

In [ ]:
%cd /content/drive/MyDrive/Rain in Australia/datasets

/content/drive/MyDrive/Rain in Australia/datasets


In [ ]:
x,y = FE(X_train_drop_miss, X_test_drop_miss, y_train_drop_miss, y_test_drop_miss) # tập dữ liệu drop feature nhiều missing values
x.to_csv('X_train_drop_miss.csv')
print(x.shape)
y.to_csv('y_train_drop_miss.csv')
x,y =FE(X_train_drop_relate, X_test_drop_relate, y_train_drop_relate, y_test_drop_relate) #tập dữ liệu drop feature dự đoán ảnh hưởng nhiều tới kq 'Humidity3pm'
x.to_csv('X_train_drop_relate.csv')
y.to_csv('y_train_drop_relate.csv')
x,y =FE(X_train_big, X_test_big, y_train_big, y_test_big) # tập dữ liệu full size
x.to_csv('X_train_big.csv')
y.to_csv('y_train_big.csv')
x,y=FE(X_train_medium, X_test_medium, y_train_medium, y_test_medium) # tập dữ liệu size medium
x.to_csv('X_train_medium.csv')
y.to_csv('y_train_medium.csv')
x,y=FE(X_train_small, X_test_small, y_train_small, y_test_small) # tập dữ liệu size small
x.to_csv('X_train_small.csv')
y.to_csv('y_train_small.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(113754, 114)


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be remov